# Random Forest Ejemplo

Implemente el algoritmo Random Forest con TensorFlow, y aplíquelo para clasificar imágenes de dígitos escritas a mano. Este ejemplo utiliza la base de datos MNIST de dígitos manuscritos como ejemplos de capacitación (http://yann.lecun.com/exdb/mnist/).

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest

# Ignora todas las GPUs, el bosque aleatorio tf no se beneficia de ello.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# Importar dstos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Parametros
num_steps = 500 # Total pasos a seguir en el entrenamiento
batch_size = 1024 # El número de muestras por lote
num_classes = 10 # Los 10 dígitos
num_features = 784 # Cada imagen tiene 28x28 píxeles
num_trees = 10
max_nodes = 1000

# Datos de entrada y de destino
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Para bosques aleatorios, las etiquetas deben ser enteras (la clase id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parametros
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

In [5]:
# Construir el Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Obtenga el gráfico de entrenamiento y las pérdidas
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Medir la precisión
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Inicializar las variables (es decir, asignar su valor por defecto) y los recursos forestales.
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'valid_leaf_threshold': 1, 'split_after_samples': 250, 'num_output_columns': 11, 'feature_bagging_fraction': 1.0, 'split_initializations_per_input': 3, 'bagged_features': None, 'min_split_samples': 5, 'max_nodes': 1000, 'num_features': 784, 'num_trees': 10, 'num_splits_to_consider': 784, 'base_random_seed': 0, 'num_outputs': 1, 'dominate_fraction': 0.99, 'max_fertile_nodes': 500, 'bagged_num_features': 784, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'num_classes': 10}
INFO:tensorflow:training graph for tree: 0
INFO:tensorflow:training graph for tree: 1
INFO:tensorflow:training graph for tree: 2
INFO:tensorflow:training graph for tree: 3
INFO:tensorflow:training graph for tree: 4
INFO:tensorflow:training graph for tree: 5
INFO:tensorflow:training graph for tree: 6
INFO:tensorflow:training graph for tree: 7
INFO:tensorflow:training graph for tree: 8
INFO:tensorflow:training graph for tr

In [6]:
# Iniciar TensorFlow session
sess = tf.train.MonitoredSession()

# Ejecutar el inicializador
sess.run(init_vars)

# Entrenamiento
for i in range(1, num_steps + 1):
    # Preparar los datos
    # Obtener el siguiente lote de datos MNIST (sólo se necesitan imágenes, no etiquetas)
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Modelo
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Step 1, Loss: -0.000000, Acc: 0.112305
Step 50, Loss: -123.800003, Acc: 0.863281
Step 100, Loss: -274.200012, Acc: 0.863281
Step 150, Loss: -425.399994, Acc: 0.872070
Step 200, Loss: -582.799988, Acc: 0.917969
Step 250, Loss: -740.200012, Acc: 0.912109
Step 300, Loss: -895.799988, Acc: 0.939453
Step 350, Loss: -998.000000, Acc: 0.924805
Step 400, Loss: -998.000000, Acc: 0.940430
Step 450, Loss: -998.000000, Acc: 0.914062
Step 500, Loss: -998.000000, Acc: 0.927734
Test Accuracy: 0.9204
